In [51]:
from pyspark.ml.classification import LogisticRegression
import findspark
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import count

In [52]:
spark = (SparkSession
         .builder
         .appName("Logistic Regression")
         .getOrCreate())

In [53]:
df = (spark.read
            .option("HEADER", True)
            .option("inferSchema", True)
            .csv("./data/MergeMatchStatsSoFifa.csv")
           )

df.show(5)

# df['home_captain'] = df['home_captain'].astype(str)

+--------+-----------+------------+-----------------+---------+---------+--------+--------+--------+-------+----------+---------------+---------------+-------------------------+----------------+--------+-------+--------+-----------------+------------------+--------+--------------+----------------+-----------------+---------------+------------+------------+-----------------+--------------+----------+---------------+------------------+-----------------+---------------+-------------+------------+---------+-------------+---------------+----------+------------+----------------+--------------+------------------------+--------------+--------+-------+--------+-----------+------------+-----------------+---------+---------+--------+--------+--------+-------+----------+---------------+---------------+-------------------------+----------------+--------+-------+--------+-----------------+------------------+--------+--------------+----------------+-------------+---------------+------------+---------

# Onehot Encoder

https://stackoverflow.com/questions/56585434/pyspark-pipeline-error-when-using-indexer-and-encoder

In [54]:
from pyspark.ml.feature import OneHotEncoder,StringIndexer,VectorAssembler

In [55]:
unusedCols = ['home_fbrefMatchId ','away_fbrefMatchId']
outputCols = ['home_score','away_score']
inputCols = [column for column in df.columns if column not in outputCols and column not in unusedCols]
encodeCols = ['home_captain','away_captain','home_manager','away_manager','home_team','away_team','away_is_home_team']




In [56]:
df[['home_captain']]

DataFrame[home_captain: string]

In [57]:
indexer = StringIndexer(inputCols=encodeCols,outputCols = [encodeCol+"_Index" for encodeCol in encodeCols])
indexerModel = indexer.fit(df)
indexer_df = indexerModel.transform(df)
indexer_df.show(5)

+--------+-----------+------------+-----------------+---------+---------+--------+--------+--------+-------+----------+---------------+---------------+-------------------------+----------------+--------+-------+--------+-----------------+------------------+--------+--------------+----------------+-----------------+---------------+------------+------------+-----------------+--------------+----------+---------------+------------------+-----------------+---------------+-------------+------------+---------+-------------+---------------+----------+------------+----------------+--------------+------------------------+--------------+--------+-------+--------+-----------+------------+-----------------+---------+---------+--------+--------+--------+-------+----------+---------------+---------------+-------------------------+----------------+--------+-------+--------+-----------------+------------------+--------+--------------+----------------+-------------+---------------+------------+---------

In [58]:
encodeer = OneHotEncoder(inputCols=[encodeCol+"_Index" for encodeCol in encodeCols],outputCols=[encodeCol+"_Onehot" for encodeCol in encodeCols])
encodeer_df = encodeer.fit(indexer_df).transform(indexer_df)
encodeer_df.show(5)

+--------+-----------+------------+-----------------+---------+---------+--------+--------+--------+-------+----------+---------------+---------------+-------------------------+----------------+--------+-------+--------+-----------------+------------------+--------+--------------+----------------+-----------------+---------------+------------+------------+-----------------+--------------+----------+---------------+------------------+-----------------+---------------+-------------+------------+---------+-------------+---------------+----------+------------+----------------+--------------+------------------------+--------------+--------+-------+--------+-----------+------------+-----------------+---------+---------+--------+--------+--------+-------+----------+---------------+---------------+-------------------------+----------------+--------+-------+--------+-----------------+------------------+--------+--------------+----------------+-------------+---------------+------------+---------

In [71]:

indexedCols = [encodeCol+"_Index" for encodeCol in encodeCols]
assembler_inputCols = encodeer_df.columns
assembler_inputCols = [assembler_inputCol for assembler_inputCol in assembler_inputCols if assembler_inputCol not in encodeCols and assembler_inputCol not in indexedCols and assembler_inputCol not in unusedCols]
assembler_df = encodeer_df.select(assembler_inputCols)
assembler = VectorAssembler(inputCols=assembler_inputCols,outputCol="features")

In [72]:
assembler.transform(assembler_df).show(5)

IllegalArgumentException: Data type string of column home_is_home_team is not supported.
Data type string of column home_possession is not supported.
Data type string of column away_is_home_team is not supported.
Data type string of column away_possession is not supported.

# Logistic Regression